In [1]:
import pandas as pd
from pandas import concat
import statsmodels.api as sm

from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels as sms
import statsmodels.tsa as tsa
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pmdarima as pm
import pandas_datareader as web
from arch import  arch_model
from math import sqrt
import numpy as np


In [2]:
from sklearn.utils import check_array
def MAPE(Y_test, preds): 
    Y_test, preds = np.array(Y_test), np.array(preds)
    return np.mean(np.abs((Y_test - preds) / Y_test)) * 100

In [ ]:
stock = web.get_data_yahoo("T",
start = "2021-01-01",
end = "2021-11-17")
y=stock['Adj Close']

Y_t=y.T
Y_t=Y_t.iloc[1:167]
Y_train=Y_t

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_var
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

In [ ]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

In [ ]:
stock = web.get_data_yahoo("NVDA",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_t=Y_t.iloc[1:978]
Y_train=Y_t
model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,12):
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,12): 
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,12):
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_var
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

In [ ]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

In [ ]:
stock = web.get_data_yahoo("MSFT",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_train=Y_t.iloc[1:978]

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_mean
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

In [ ]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

In [3]:
stock = web.get_data_yahoo("GOOGL",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_train=Y_t.iloc[1:978]

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_mean
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=9200.489, Time=1.24 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=9224.325, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=9216.687, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=9216.861, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=9227.140, Time=0.02 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=9212.861, Time=1.26 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=9212.889, Time=1.19 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.59 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=9202.691, Time=1.53 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=9220.303, Time=0.78 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=9223.329, Time=1.28 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=9222.197, Time=1.29 sec
 ARIMA(4,1,4)(0,0,0)[0] intercept   : AIC=9193.419, Time=1.68 sec
 ARIMA(5,1,4)(0,0,0)[0] intercept   : AIC=9195.688, Time=1.86 sec
 ARIMA(4,1,5)(0,0,0)[0] intercept   : 

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
e

...

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 5037.195633303527
Iteration:      2,   Func. Count:     40,   Neg. LLF: 4943.7345672053725
Iteration:      3,   Func. Count:     61,   Neg. LLF: 4669.776456716438
Iteration:      4,   Func. Count:     82,   Neg. LLF: 4537.853426911146
Iteration:      5,   Func. Count:    102,   Neg. LLF: 4530.761118002161
Iteration:      6,   Func. Count:    122,   Neg. LLF: 4546.762830282892
Iteration:      7,   Func. Count:    142,   Neg. LLF: 4532.316040249442
Iteration:      8,   Func. Count:    162,   Neg. LLF: 4510.717812723296
Iteration:      9,   Func. Count:    182,   Neg. LLF: 4493.774496884471
Iteration:     10,   Func. Count:    202,   Neg. LLF: 4488.114758729335
Iteration:     11,   Func. Count:    221,   Neg. LLF: 4489.405698139522
Iteration:     12,   Func. Count:    241,   Neg. LLF: 4490.113043678344
Iteration:     13,   Func. Count:    261,   Neg. LLF: 4487.145218654177
Iteration:     14,   Func. Count:    280,   Neg. LLF: 4487.4813

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 5063.344812169854
Iteration:      2,   Func. Count:     40,   Neg. LLF: 5009.52025243899
Iteration:      3,   Func. Count:     61,   Neg. LLF: 4879.453071296523
Iteration:      4,   Func. Count:     82,   Neg. LLF: 4873.450280804909
Iteration:      5,   Func. Count:    103,   Neg. LLF: 4529.017439176714
Iteration:      6,   Func. Count:    123,   Neg. LLF: 4509.77285149823
Iteration:      7,   Func. Count:    143,   Neg. LLF: 4640.471067168263
Iteration:      8,   Func. Count:    163,   Neg. LLF: 4520.891362819985
Iteration:      9,   Func. Count:    183,   Neg. LLF: 4516.682204947141
Iteration:     10,   Func. Count:    203,   Neg. LLF: 4502.696717362408
Iteration:     11,   Func. Count:    223,   Neg. LLF: 4499.32047438267
Iteration:     12,   Func. Count:    243,   Neg. LLF: 4509.470316637829
Iteration:     13,   Func. Count:    263,   Neg. LLF: 4493.431165408309
Iteration:     14,   Func. Count:    282,   Neg. LLF: 4495.28104836

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [4]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

1.062144541346719
58.010549782199256
1.060648333493007
